# Databricks Lakebase Inventory - BI Dashboard Deployment

This notebook automates the deployment of the Lakeview Dashboard for the Lakebase Inventory Management System.

## What This Notebook Does:
1. Creates a managed database catalog in the Lakebase instance
2. Updates the dashboard template with the parameterized catalog name
3. Creates or updates a Databricks Lakeview Dashboard
4. Updates the app.yaml file with the dashboard ID
5. Redeploys the Databricks App

## Prerequisites:
- Lakebase database instance already created (from DeploymentAutomation.ipynb)
- Databricks App already deployed
- Unity Catalog enabled workspace
- Appropriate permissions to create catalogs and dashboards


## Step 0: Install Required Libraries


In [0]:
%pip install databricks-sdk --upgrade
dbutils.library.restartPython()


## Configuration Section

**IMPORTANT**: Update these configuration values to match your deployment!


In [0]:
# ============= CONFIGURATION - EDIT THESE VALUES =============

# Lakebase Database Configuration
LAKEBASE_DB_NAME = "inventory-lakebase-db"  # Name of your Lakebase database instance
LOGICAL_DATABASE_NAME = "inventory_app"  # Logical database name within Lakebase

# Catalog Configuration
CATALOG_NAME = "reynoldspravindev_inventory_live"  # Unity Catalog name for your Lakebase data
CREATE_CATALOG_IF_NOT_EXISTS = True  # Set to False if catalog already exists

# Dashboard Configuration
DASHBOARD_DISPLAY_NAME = "Lakebase Inventory Dashboard - Reynolds"  # Display name for the dashboard
DASHBOARD_TEMPLATE_PATH = "dashboards/Lakebase Inventory Dashboard - Reynolds.lvdash.json"  # Path to dashboard template
WAREHOUSE_ID = None  # Leave None to use default warehouse, or specify a warehouse ID

# App Configuration (must match your DeploymentAutomation.ipynb settings)
APP_NAME = "inventory-management-app"  # Name of your Databricks App
REPO_WORKSPACE_PATH = "/Repos/Production/databricks-lakebase-inventory"  # Workspace path for repo

# Schema Configuration (must match your app.py settings)
POSTGRES_SCHEMA = "inventory_app"  # Schema name in Lakebase

# ==============================================================


## Initialize Databricks Workspace Client


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseCatalog
from databricks.sdk.service.apps import AppDeployment
from databricks.sdk.service.workspace import ImportFormat
import json
import re
from datetime import timedelta

# Initialize workspace client
w = WorkspaceClient()

# Get current workspace info
current_user = w.current_user.me()

# Get the correct workspace URL
try:
    workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
    workspace_url = f"https://{workspace_url}"
except:
    workspace_url = w.config.host

print(f"✓ Connected to workspace: {workspace_url}")
print(f"✓ Current user: {current_user.user_name}")
print(f"✓ User ID: {current_user.id}")


## Step 1: Create Managed Database Catalog

This creates a Unity Catalog catalog that connects to your Lakebase database instance.


In [0]:
print("Step 1: Creating Managed Database Catalog...\n")

try:
    # Create the database catalog
    catalog = DatabaseCatalog(
        name=CATALOG_NAME,
        database_instance_name=LAKEBASE_DB_NAME,
        database_name=LOGICAL_DATABASE_NAME,
        create_database_if_not_exists=CREATE_CATALOG_IF_NOT_EXISTS
    )
    
    print(f"Creating catalog '{CATALOG_NAME}' for database instance '{LAKEBASE_DB_NAME}'...")
    
    # Create the catalog
    created_catalog = w.database.create_database_catalog(catalog=catalog)
    
    print(f"✓ Database catalog '{CATALOG_NAME}' created successfully!")
    print(f"  Catalog Name: {created_catalog.name}")
    print(f"  Database Instance: {created_catalog.database_instance_name}")
    print(f"  Logical Database: {created_catalog.database_name}")
    
except Exception as e:
    # Check if catalog already exists
    if "already exists" in str(e).lower() or "AlreadyExists" in str(e):
        print(f"⚠ Catalog '{CATALOG_NAME}' already exists. Using existing catalog.")
        try:
            existing_catalog = w.database.get_database_catalog(name=CATALOG_NAME)
            print(f"  Catalog Name: {existing_catalog.name}")
            print(f"  Database Instance: {existing_catalog.database_instance_name}")
            print(f"  Logical Database: {existing_catalog.database_name}")
        except Exception as get_error:
            print(f"  Note: Could not retrieve catalog details: {get_error}")
    else:
        print(f"✗ Error creating database catalog: {e}")
        raise

print(f"\n" + "="*60)


## Step 2: Load and Parameterize Dashboard Template

This loads the dashboard template and replaces the catalog name with your configured value.


In [0]:
print("Step 2: Loading and Parameterizing Dashboard Template...\n")

try:
    # Read the dashboard template from the workspace
    template_path = f"{REPO_WORKSPACE_PATH}/{DASHBOARD_TEMPLATE_PATH}"
    
    print(f"Reading dashboard template from: {template_path}")
    
    # Download the file from workspace
    file_info = w.workspace.download(path=template_path)
    dashboard_template = json.loads(file_info.contents.read())
    
    print(f"✓ Dashboard template loaded successfully")
    
    # Convert template to JSON string for regex replacement
    dashboard_json_str = json.dumps(dashboard_template, indent=2)
    
    # Find the current catalog name in the template (pattern: catalog_name.schema_name.table_name)
    # We'll look for patterns like "reynoldspravindev_inventory_live.inventory_app.table_name"
    catalog_pattern = r'([a-zA-Z0-9_]+)\.inventory_app\.'
    
    # Find what catalog is currently in the template
    match = re.search(catalog_pattern, dashboard_json_str)
    if match:
        old_catalog = match.group(1)
        print(f"  Found existing catalog in template: {old_catalog}")
        
        # Replace all occurrences of the old catalog with the new one
        dashboard_json_str = re.sub(
            rf'{old_catalog}\.{POSTGRES_SCHEMA}\.', 
            f'{CATALOG_NAME}.{POSTGRES_SCHEMA}.', 
            dashboard_json_str
        )
        
        print(f"  Replaced '{old_catalog}' with '{CATALOG_NAME}'")
    else:
        print(f"  ⚠ Could not find catalog pattern in template. Using template as-is.")
    
    # Convert back to JSON object
    dashboard_definition = json.loads(dashboard_json_str)
    
    print(f"✓ Dashboard template parameterized successfully")
    print(f"  Catalog: {CATALOG_NAME}")
    print(f"  Schema: {POSTGRES_SCHEMA}")
    
except Exception as e:
    print(f"✗ Error loading/parameterizing dashboard template: {e}")
    raise

print(f"\n" + "="*60)


## Step 3: Create or Update Databricks Lakeview Dashboard

This creates a new dashboard or updates an existing one if a dashboard ID is provided in app.yaml.


In [0]:
print("Step 3: Creating/Updating Databricks Lakeview Dashboard...\n")

try:
    # Check if dashboard ID exists in app.yaml
    app_yaml_path = f"{REPO_WORKSPACE_PATH}/app.yaml"
    
    try:
        app_yaml_content = w.workspace.download(path=app_yaml_path).contents.read().decode('utf-8')
        
        # Extract DASHBOARD_ID from app.yaml
        dashboard_id_match = re.search(r"DASHBOARD_ID['\"]?\s*\n\s*value:\s*['\"]([^'\"]+)['\"]?", app_yaml_content)
        existing_dashboard_id = dashboard_id_match.group(1) if dashboard_id_match and dashboard_id_match.group(1) else None
        
        if existing_dashboard_id and existing_dashboard_id.strip():
            print(f"Found existing dashboard ID in app.yaml: {existing_dashboard_id}")
        else:
            existing_dashboard_id = None
            print("No dashboard ID found in app.yaml. Will create new dashboard.")
    except:
        existing_dashboard_id = None
        print("Could not read app.yaml. Will create new dashboard.")
    
    # Determine warehouse ID
    if WAREHOUSE_ID is None:
        # Get the first available SQL warehouse
        warehouses = list(w.warehouses.list())
        if warehouses:
            warehouse_id = warehouses[0].id
            print(f"Using warehouse: {warehouses[0].name} (ID: {warehouse_id})")
        else:
            raise Exception("No SQL warehouses found. Please create one or specify WAREHOUSE_ID.")
    else:
        warehouse_id = WAREHOUSE_ID
        print(f"Using specified warehouse ID: {warehouse_id}")
    
    # Create or update dashboard using Lakeview API
    if existing_dashboard_id:
        # Update existing dashboard
        print(f"\nUpdating existing dashboard '{existing_dashboard_id}'...")
        
        try:
            # Get the current dashboard
            current_dashboard = w.lakeview.get(dashboard_id=existing_dashboard_id)
            
            # Update the dashboard
            updated_dashboard = w.lakeview.update(
                dashboard_id=existing_dashboard_id,
                display_name=DASHBOARD_DISPLAY_NAME,
                serialized_dashboard=json.dumps(dashboard_definition),
                warehouse_id=warehouse_id
            )
            
            # Publish the dashboard
            published = w.lakeview.publish(dashboard_id=existing_dashboard_id)
            
            dashboard_id = existing_dashboard_id
            print(f"✓ Dashboard updated and published successfully!")
            
        except Exception as update_error:
            print(f"⚠ Could not update existing dashboard: {update_error}")
            print(f"Creating new dashboard instead...")
            existing_dashboard_id = None
    
    if not existing_dashboard_id:
        # Create new dashboard
        print(f"\nCreating new dashboard '{DASHBOARD_DISPLAY_NAME}'...")
        
        # Create the dashboard
        new_dashboard = w.lakeview.create(
            display_name=DASHBOARD_DISPLAY_NAME,
            serialized_dashboard=json.dumps(dashboard_definition),
            warehouse_id=warehouse_id,
            parent_path=f"/Users/{current_user.user_name}"
        )
        
        dashboard_id = new_dashboard.dashboard_id
        
        # Publish the dashboard
        published = w.lakeview.publish(dashboard_id=dashboard_id)
        
        print(f"✓ Dashboard created and published successfully!")
    
    print(f"  Dashboard ID: {dashboard_id}")
    print(f"  Dashboard URL: {workspace_url}/sql/dashboardsv3/{dashboard_id}")
    
    # Store dashboard ID for next step
    CREATED_DASHBOARD_ID = dashboard_id
    
except Exception as e:
    print(f"✗ Error creating/updating dashboard: {e}")
    print(f"\nNote: Make sure you have permissions to create dashboards and that the warehouse is running.")
    raise

print(f"\n" + "="*60)


## Step 4: Update app.yaml with Dashboard ID

This updates the app.yaml file with the dashboard ID so the app can embed the dashboard.


In [0]:
print("Step 4: Updating app.yaml with Dashboard ID...\n")

try:
    # Read current app.yaml
    app_yaml_path = f"{REPO_WORKSPACE_PATH}/app.yaml"
    app_yaml_content = w.workspace.download(path=app_yaml_path).contents.read().decode('utf-8')
    
    print(f"Reading app.yaml from: {app_yaml_path}")
    
    # Update DASHBOARD_ID value
    # Pattern matches: - name: 'DASHBOARD_ID'\n    value: 'old_value'
    dashboard_pattern = r"(- name: ['\"]DASHBOARD_ID['\"]?\s*\n\s*value: ['\"]?)([^'\"]*)([\'\"]?)"
    
    if re.search(dashboard_pattern, app_yaml_content):
        # Replace existing dashboard ID
        updated_yaml = re.sub(
            dashboard_pattern,
            rf"\g<1>{CREATED_DASHBOARD_ID}\g<3>",
            app_yaml_content
        )
        print(f"  Updated existing DASHBOARD_ID value")
    else:
        # Add DASHBOARD_ID if it doesn't exist
        updated_yaml = app_yaml_content.rstrip() + f"\n  - name: 'DASHBOARD_ID'\n    value: '{CREATED_DASHBOARD_ID}'\n"
        print(f"  Added DASHBOARD_ID to app.yaml")
    
    # Write updated app.yaml back to workspace
    w.workspace.upload(
        path=app_yaml_path,
        content=updated_yaml.encode('utf-8'),
        format=ImportFormat.AUTO,
        overwrite=True
    )
    
    print(f"✓ app.yaml updated successfully!")
    print(f"  Dashboard ID: {CREATED_DASHBOARD_ID}")
    
except Exception as e:
    print(f"✗ Error updating app.yaml: {e}")
    print(f"\nPlease manually update app.yaml with:")
    print(f"  - name: 'DASHBOARD_ID'")
    print(f"    value: '{CREATED_DASHBOARD_ID}'")
    raise

print(f"\n" + "="*60)


## Step 5: Redeploy Databricks App

This redeploys the Databricks App with the updated app.yaml configuration.


In [0]:
print("Step 5: Redeploying Databricks App...\n")

try:
    print(f"Redeploying app '{APP_NAME}' with updated configuration...")
    
    # Create deployment configuration
    app_deployment = AppDeployment(
        source_code_path=REPO_WORKSPACE_PATH
    )
    
    # Deploy the app
    deployment = w.apps.deploy_and_wait(
        app_name=APP_NAME,
        app_deployment=app_deployment,
        timeout=timedelta(minutes=20)
    )
    
    print(f"✓ App redeployment completed!")
    print(f"  Deployment ID: {deployment.deployment_id}")
    print(f"  Status: {deployment.status}")
    
    # Get app status and URL
    app_status = w.apps.get(name=APP_NAME)
    if hasattr(app_status, 'url') and app_status.url:
        print(f"  App URL: {app_status.url}")
        print(f"\n✓ Your app is now deployed with the embedded dashboard!")
        print(f"\nAccess your app at: {app_status.url}")
        print(f"View the dashboard at: {workspace_url}/sql/dashboardsv3/{CREATED_DASHBOARD_ID}")
    
except Exception as e:
    print(f"✗ Error redeploying app: {e}")
    print(f"\nYou can manually redeploy using the command in the cell below.")

print(f"\n" + "="*60)


## 🎉 Deployment Complete!

Your BI dashboard has been successfully deployed! Here's what was created:

1. ✅ **Database Catalog**: Unity Catalog catalog connected to your Lakebase instance
2. ✅ **Lakeview Dashboard**: Interactive dashboard with your inventory analytics
3. ✅ **App Configuration**: Updated app.yaml with dashboard ID
4. ✅ **App Redeployment**: Your app is now running with the embedded dashboard

### Next Steps:

- Access your app and navigate to the Dashboard page to see the embedded analytics
- Customize the dashboard by editing it in the Databricks SQL UI
- Share the dashboard with your team using Databricks permissions

### Troubleshooting:

If the dashboard doesn't appear in your app:
1. Verify the dashboard ID in app.yaml matches the created dashboard
2. Ensure your app has permissions to access the SQL warehouse
3. Check that the catalog and schema names match your Lakebase configuration
4. Manually redeploy using the command below if needed


---
## Manual Redeployment Command

If you need to redeploy the app manually, run this cell:


In [0]:
# Manual redeployment command
from databricks.sdk.service.apps import AppDeployment
from datetime import timedelta

print(f"Manually redeploying app '{APP_NAME}'...\n")

app_deployment = AppDeployment(source_code_path=REPO_WORKSPACE_PATH)
deployment = w.apps.deploy_and_wait(
    app_name=APP_NAME,
    app_deployment=app_deployment,
    timeout=timedelta(minutes=20)
)

print(f"✓ Redeployment completed!")
print(f"  Deployment ID: {deployment.deployment_id}")
print(f"  Status: {deployment.status}")

app_status = w.apps.get(name=APP_NAME)
if hasattr(app_status, 'url') and app_status.url:
    print(f"  App URL: {app_status.url}")


---
## Verification Commands

Run these cells to verify your deployment:


In [0]:
# Verify catalog exists
print("Verifying catalog...\n")
try:
    catalog_info = w.database.get_database_catalog(name=CATALOG_NAME)
    print(f"✓ Catalog '{CATALOG_NAME}' exists")
    print(f"  Database Instance: {catalog_info.database_instance_name}")
    print(f"  Logical Database: {catalog_info.database_name}")
except Exception as e:
    print(f"✗ Catalog verification failed: {e}")


In [0]:
# Verify dashboard exists
print("Verifying dashboard...\n")
try:
    dashboard_info = w.lakeview.get(dashboard_id=CREATED_DASHBOARD_ID)
    print(f"✓ Dashboard '{dashboard_info.display_name}' exists")
    print(f"  Dashboard ID: {dashboard_info.dashboard_id}")
    print(f"  Path: {dashboard_info.path}")
    print(f"  URL: {workspace_url}/sql/dashboardsv3/{dashboard_info.dashboard_id}")
except Exception as e:
    print(f"✗ Dashboard verification failed: {e}")


In [0]:
# Verify app deployment
print("Verifying app deployment...\n")
try:
    app_info = w.apps.get(name=APP_NAME)
    print(f"✓ App '{APP_NAME}' is deployed")
    if hasattr(app_info, 'url') and app_info.url:
        print(f"  App URL: {app_info.url}")
    if hasattr(app_info, 'status'):
        print(f"  Status: {app_info.status}")
    
    # Check app.yaml for dashboard ID
    app_yaml_path = f"{REPO_WORKSPACE_PATH}/app.yaml"
    app_yaml_content = w.workspace.download(path=app_yaml_path).contents.read().decode('utf-8')
    if CREATED_DASHBOARD_ID in app_yaml_content:
        print(f"  ✓ Dashboard ID is configured in app.yaml")
    else:
        print(f"  ⚠ Dashboard ID not found in app.yaml")
        
except Exception as e:
    print(f"✗ App verification failed: {e}")
